# <span style="color:#F72585"><center>Modelo seq2seq</center></span>

<center>Construcción de un traductor</center>

<figure>
<center>
<img src="../Imagenes/writing.jpg" width="700" height="600" align="center"/>
</center>
</figure>

Fuente: [Pixabay](https://pixabay.com/es/photos/escribiendo-conjunto-de-plomo-705667/)

## <span style="color:#4361EE">Introducción</span>

El modelo de secuencia a secuencia (*seq2seq*)  es un modelo de aprendizaje que convierte una secuencia de entrada en una secuencia de salida. 

En este contexto, la secuencia es una lista de símbolos, correspondiente a las palabras en una oración. 

El modelo seq2seq ha logrado un gran éxito en campos como la traducción automática, los sistemas de diálogo, la respuesta a preguntas y el resumen de texto. 

Todas estas tareas pueden considerarse como la tarea de aprender un modelo que convierte una secuencia de entrada en una secuencia de salida.

La imagen muestra la arquitectura general del modelo.

<figure>
<center>
<img src="../Imagenes/seq2seq.png" width="500" height="400" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Arquitectura del modelo seq2seq</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

El modelo consta esencialmente de las siguientes capas.

1. Capa de incrustación del codificador
2. Capa recurrente del codificador
3. Capa de incrustación del decodificador
4. Capa recurrente del decodificador
5. Capa de salida del decodificador

Veamos la implementación en t.keras propuesta por F. Collet, [Character-level recurrent sequence-to-sequence model](https://keras.io/examples/nlp/lstm_seq2seq/).

## <span style="color:#4361EE">Importa módulos</span>


In [2]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras

from tensorflow.keras.layers import Input, LSTM, Dense

from tensorflow.keras.models import Model

from tensorflow.keras.utils import plot_model

## <span style="color:#4361EE">Descarga los datos</span>

Puede descargar los de [Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/)

## <span style="color:#4361EE">Configuración</span>


In [3]:
batch_size = 64 # tamaño de los lotes para entrenamiento
epochs = 100 # número de epochs
latent_dim = 256 # dimensión del espacio latente para el encoder
num_samples = 10000
# path del archivo
data_path = "../Datos/spa-eng/spa.txt" # 124548 lines

## <span style="color:#4361EE">Prepara los datos</span>

In [4]:
# Vectoriza los datos
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n") # 124548 lines

for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, target_text, _ = line.split("\t")
    # Usaremos "tab" como el  caracter de inicio (start sequence)
    # para los targets, y "\n" como el caracter de fin de secuencia "end sequence"
    target_text = "\t" + target_text + "\n"
    # sube las líneas a  las listas
    input_texts.append(input_text)
    target_texts.append(target_text)
    # completa los conjuntos de caracteres si es necesario
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# Convierte los dos conjuntos de caracteres
# en dos listas ordenadas
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))  
# calcule el número de tokens (caracteres) en ambos lados
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# calcula la máxima longitud de las secuencias en cada lado
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("preparando datos...")
# crea diccionarios de tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# crea los tensores one-hot para el encoder y el decoder
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print ("....\ndatos preparados")    

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 84
Max sequence length for inputs: 16
Max sequence length for outputs: 42
preparando datos...
....
datos preparados


## <span style="color:#4361EE">Construye el modelo</span>

### <span style="color:#4CC9F0">Encoder</span>

In [6]:
# define una secuencia de entrada y la procesa
encoder_inputs = Input(shape = (None, num_encoder_tokens))

# capa recurrente del encoder
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Descartamos las salidas (encoder_outputs)
# solamente se conserva las memoria de  corto (state_h) y 
# largo plazo(state_c)
encoder_states = [state_h, state_c]

### <span style="color:#4CC9F0">Decoder</span>

In [7]:
# Configuramos el decoder, usando 'encoder_states' como estado inicial
decoder_inputs = Input(shape= (None, num_decoder_tokens))

# capa recurrente del decoder
# Configuramos nuestro decodificador para devolver secuencias de salida completas,
# y también para devolver estados internos. No usamos los
# estados retornados en el modelo de entrenamiento, pero los usaremos en inferencia.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### <span style="color:#4CC9F0">Modelo completo</span>

In [8]:
# Definir el modelo que se convertirá
# `encoder_input_data` & `decoder_input_data` dentro de `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [9]:
model.summary()
plot_model(model, to_file='../Imagenes/s2s.png', 
           show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, 69)]   0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None, 84)]   0           []                               
                                                                                                  
 lstm_1 (LSTM)                  [(None, 256),        333824      ['input_2[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

## <span style="color:#4361EE">Entrena el modelo</span>

In [10]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 34s 242ms/step - loss: 1.5100 - accuracy: 0.6595 - val_loss: 1.3299 - val_accuracy: 0.6260
Epoch 2/100
125/125 [==============================] - 29s 228ms/step - loss: 1.0610 - accuracy: 0.7075 - val_loss: 1.1347 - val_accuracy: 0.6888
Epoch 3/100
125/125 [==============================] - 28s 226ms/step - loss: 0.8960 - accuracy: 0.7501 - val_loss: 0.9909 - val_accuracy: 0.7154
Epoch 4/100
125/125 [==============================] - 28s 227ms/step - loss: 0.8002 - accuracy: 0.7660 - val_loss: 0.9136 - val_accuracy: 0.7312
Epoch 5/100
125/125 [==============================] - 28s 226ms/step - loss: 0.7477 - accuracy: 0.7775 - val_loss: 0.8647 - val_accuracy: 0.7414
Epoch 6/100
125/125 [==============================] - 28s 225ms/step - loss: 0.7081 - accuracy: 0.7886 - val_loss: 0.8303 - val_accuracy: 0.7517
Epoch 7/100
125/125 [==============================] - 28s 228ms/step - loss: 0.6771 - accuracy: 0.7961 - val_loss: 0.7997 -

In [20]:
# Guardar el Modelo
#model.save("../Saved_Models/s2s")

In [ ]:
# Recrea exactamente el mismo modelo solo desde el archivo
#new_model = keras.models.load_model('../Saved_Models/s2s')

## <span style="color:#4361EE">Modelo de inferencia</span>

1. Codifica la entrada y recuperar el estado inicial del decodificador.
2. Ejecuta un paso del decodificador con este estado inicial y un token de "inicio de secuencia" como objetivo. La salida será el próximo token de destino.
3. Repite con el token de destino actual y los estados actuales.

In [18]:
#
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_4")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_5")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Índice de token de búsqueda inversa para decodificar secuencias nuevamente
# algo legible
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Codifique la entrada como vectores de estado.
    states_value = encoder_model.predict(input_seq)

    # Genere una secuencia de destino vacía de longitud 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Rellene el primer carácter de la secuencia de destino con el carácter de inicio.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Bucle de muestreo para un lote de secuencias
    # (para simplificar, aquí asumimos un lote de tamaño 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Muestra un token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        #Condición de salida: alcanzar la longitud máxima
        # o encontrar el carácter de parada.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Actualice la secuencia de destino (de longitud 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Actualizar estados
        states_value = [h, c]
    return decoded_sentence

## <span style="color:#4361EE">Prueba del modelo</span>

Ahora podemos generar oraciones decodificadas como tales:

In [19]:
for seq_index in range(100):
    # Tome una secuencia (parte del conjunto de entrenamiento)
    # para probar la decodificación.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Oración de entrada:", input_texts[seq_index])
    print("Oración decodificada:", decoded_sentence)

1/1 [==============================] - 0s 24ms/step
-
Oración de entrada: Go.
Oración decodificada: Ve.

1/1 [==============================] - 0s 24ms/step
-
Oración de entrada: Go.
Oración decodificada: Ve.

1/1 [==============================] - 0s 23ms/step
-
Oración de entrada: Go.
Oración decodificada: Ve.

1/1 [==============================] - 0s 26ms/step
-
Oración de entrada: Go.
Oración decodificada: Ve.

1/1 [==============================] - 0s 24ms/step
-
Oración de entrada: Hi.
Oración decodificada: Hola.

1/1 [==============================] - 0s 24ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corred!

1/1 [==============================] - 0s 24ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corred!

1/1 [==============================] - 0s 25ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corred!

1/1 [==============================] - 0s 24ms/step
-
Oración de entrada: Run!
Oración decodificada: ¡Corred!

1/1 [========================

## <span style="color:#4361EE">Referencias</span>

1. Basado en F. Collet, [Character-level recurrent sequence-to-sequence model](https://keras.io/examples/nlp/lstm_seq2seq/)
2. [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)
3. Ilya Sutskever et al. (Google),[Sequence to Sequence Learning
with Neural Network](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)